<a href="https://colab.research.google.com/github/ryganon/projetos_IAI/blob/master/Sistemas_Multi_Agentes/Agentes_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuração

In [ ]:
!pip install backtrader

In [85]:
import matplotlib as plt
import pandas as pd
import numpy as np
import backtrader as bt

In [5]:
dados = pd.read_csv("acoes_indice_bovespa_indicadores.csv")

In [32]:
dados["DATAPREGAO"] = pd.to_datetime(dados['DATAPREGAO'])

# Modelagem dos agentes

* Datas: https://towardsdatascience.com/working-with-datetime-in-pandas-dataframe-663f7af6c587
* Criando SMA ou EMA: https://towardsdatascience.com/making-a-trade-call-using-simple-moving-average-sma-crossover-strategy-python-implementation-29963326da7a

In [156]:
class BrokerAgent:
    def __init__(self, data):
        self.stocks = data
        self.global_stocks = {item: 10000 for item in list(self.stocks["CODNEG"].unique())}

    def get_stocks(self):
        return self.stocks

    def get_stocks_by_date(self, ref_date, selected_stocks):
        #return self.stocks.loc[self.stocks["DATAPREGAO"] <= ref_date]

        temp = self.stocks.loc[self.stocks["DATAPREGAO"] <= ref_date]
        return temp.loc[temp["CODNEG"].isin(selected_stocks)]

    def get_global_stocks(self):
        return self.global_stocks    

In [253]:
class TradingAgent:
    def __init__(self, broker):
        # agente broker
        self.broker = broker

        # inicializa portfólio
        self.stocks = {item: 0 for item in list(self.broker.get_stocks()["CODNEG"].unique())}

        # valor de investimento inicial
        self.trading_amount = 10000

        # valor total da carteira
        self.portfolio_profit = {item: 0 for item in list(self.broker.get_stocks()["CODNEG"].unique())}

        
    def working_date(self, ref_date):
        self.ref_date = ref_date

    def working_stocks(self, selected_stocks):
        self.selected_stocks = selected_stocks

    def get_stocks(self):
        return self.broker.get_stocks_by_date(self.ref_date, self.selected_stocks)


class TradingAgentSimple(TradingAgent):

    def buy_stock(self, buy_data, qty):
        final_price = buy_data["PREULT"] * qty

        # se houver dinheiro
        if final_price < self.trading_amount:

            # adiciona qtde no portifolio
            self.stocks[buy_data["CODNEG"]] += qty 
            # desconta o valor da compra do amount
            self.trading_amount -= final_price 

            # atualiza a carteira
            # valor = total de acoes no portfolio * valor de ultima negociacao
            self.portfolio_profit[buy_data["CODNEG"]] = self.stocks[buy_data["CODNEG"]] * buy_data["PREULT"]
            
            print('{} \t < COMPRA > \t {} \t SMA7 = {:.2f} \t SMA14 = {:.2f}'.format(buy_data["DATAPREGAO"], buy_data["CODNEG"], buy_data["SMA7"], buy_data["SMA14"]))
        

    def sell_stock(self, sell_data, qty):
        final_price = sell_data["PREULT"] * qty

        # remove qtde no portifolio
        self.stocks[sell_data["CODNEG"]] -= qty 
        
        # retorna o valor da venda do amount
        self.trading_amount += final_price 

        # atualiza a carteira
        # valor = total de acoes no portfolio * valor de ultima negociacao
        self.portfolio_profit[sell_data["CODNEG"]] = self.stocks[sell_data["CODNEG"]] * sell_data["PREULT"]
            
        print('{} \t < VENDA > \t {} \t SMA7 = {:.2f} \t SMA14 = {:.2f}'.format(sell_data["DATAPREGAO"], sell_data["CODNEG"], sell_data["SMA7"], sell_data["SMA14"]))
    

    # cruzamento de medias moveis
    def main_strategy(self):
        # pegando os dados do papel
        hist_data = self.get_stocks()
        
        # verificando o valor atual da ação 
        temp_data = hist_data.loc[hist_data["DATAPREGAO"] == self.ref_date]

        for index, row in temp_data.iterrows():
            if row["SMA7"] > row["SMA14"]:
                # executa compra
                self.buy_stock(row, 100)
            else:
                self.sell_stock(row, 100)
            
            

        

# Testando os Agentes

In [158]:
broker = BrokerAgent(dados)

In [254]:
# Criando um Agente Simples
simple_agent = TradingAgentSimple(broker)

# Definindo as ações que serão negociadas
simple_agent.working_stocks(["MGLU3", "VVAR3"])

print("Valor disponível para investimentos:", simple_agent.trading_amount)
print("Valorização da carteira:", sum(simple_agent.portfolio_profit.values()))

# Data inicial da simulação
start_at = pd.to_datetime("2019-10-20")
# Data final da simulação
end_at = pd.to_datetime("2019-10-25")

print("\nINICIANDO SIMULAÇÃO \n")
print("Data - Tipo Op. - Sigla - SMA7 - SMA14 \n")

# inicia a simulação
for trading_day in list(pd.date_range(start_at, end_at)):
    simple_agent.working_date(trading_day)
    simple_agent.main_strategy()

print("\nFIM SIMULAÇÃO \n")

print("Valor disponível para investimentos:", simple_agent.trading_amount)
print("Valor da carteira:", sum(simple_agent.portfolio_profit.values()))
print("Valorização total: ", sum(simple_agent.portfolio_profit.values()) + simple_agent.trading_amount)

Valor disponível para investimentos: 10000
Valorização da carteira: 0

INICIANDO SIMULAÇÃO 

Data - Tipo Op. - Sigla - SMA7 - SMA14 

2019-10-21 00:00:00 	 < COMPRA > 	 MGLU3 	 SMA7 = 42.36 	 SMA14 = 40.29
2019-10-21 00:00:00 	 < COMPRA > 	 VVAR3 	 SMA7 = 7.81 	 SMA14 = 7.75
2019-10-22 00:00:00 	 < COMPRA > 	 MGLU3 	 SMA7 = 42.82 	 SMA14 = 40.81
2019-10-25 00:00:00 	 < VENDA > 	 VVAR3 	 SMA7 = 7.70 	 SMA14 = 7.71

FIM SIMULAÇÃO 

Valor disponível para investimentos: 1280.0
Valor da carteira: 8744.0
Valorização total:  10024.0


In [255]:
for acao in simple_agent.selected_stocks:
    print(acao,  "QTDE:", simple_agent.stocks[acao], "VALOR TOTAL:", simple_agent.portfolio_profit[acao])

MGLU3 QTDE: 200 VALOR TOTAL: 8744.0
VVAR3 QTDE: 0 VALOR TOTAL: 0.0
